### AI프로젝트입문 (ECE30007-02)
Spring, 2023

Team Project: 미드바르 뿌리 이미지 인식 챌린지

`midbarClassification.py`

Author: 이현서 (ID#22100600) <hslee@handong.ac.kr>


### Modules

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL.Image as Image
import numpy as np
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

### Train

In [ ]:
EPOCHS = 50
BATCH_SIZES = 16
LEARNING_RATE = 0.001

CLASSES = ["0012", "1224", "2436", "3648", "4860", "6072", "OVER"]
NUM_CLASSES = len(CLASSES)

DATASET_PATH = "./Dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
TEST_PATH = os.path.join(DATASET_PATH, "test")

writer = SummaryWriter()

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
class midbarDataset(data.Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.file_list = []
        for root, dirs, files in os.walk(img_dir):
            for file in files:
                if file.endswith(".png"):
                    self.file_list.append(os.path.join(root, file))


    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        image = Image.open(img_path).convert("L")

        label = 0

        for i in range(len(img_path) - 1, 0, -1):
            if img_path[i] == "/":
                time = int(img_path[i+1:-4])
                
                if time > 259200:
                    label = 6
                elif time > 216000:
                    label = 5
                elif time > 172800:
                    label = 4
                elif time > 129600:
                    label = 3
                elif time > 86400:
                    label = 2
                elif time > 43200:
                    label = 1
                else:
                    label = 0

                break

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
train_dataset = midbarDataset(TRAIN_PATH, transform=train_transform)
test_dataset = midbarDataset(TEST_PATH, transform=test_transform)

train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZES, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZES, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU support : " + ("True" if torch.cuda.is_available() else "False"))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, 7)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        x = x.view(-1, 128 * 16 * 16)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)
        
        return x
    
net = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [ ]:
total_loss = 0
for epoch in range(EPOCHS):
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/EPOCH", loss, epoch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (i + 1) % 10 == 0:
            avg_loss = total_loss / 10
            print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(train_loader)}], Loss: {avg_loss:.4f}")
            total_loss = 0

print("Training finished.")

writer.close()

torch.save(net.state_dict(), "./midbar.pth")

### Evaluation

In [ ]:
net.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")